# DLMI challenge notebook 

## Firas Azzouz and Hamza Gharbi

In the present notebook you can find our full code to generate our "best" submission on public leaderboard.

First we import the necessary packages.

In [ ]:
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline
import os 
from PIL import Image
import numpy as np
from datetime import datetime
from torch.utils.data import DataLoader,Dataset
import torch
import torch.nn as nn
import torchvision.models as models
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from sklearn.metrics import balanced_accuracy_score , average_precision_score , accuracy_score
import numpy as np
from torchvision import transforms
import pickle



In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


# Download data 

To download data , we use google_drive_downloader package to get the zip file that we already uploaded on our google drive account. Normally we created an open link so it shoud be  downloaded without errors to your colab session.

In [ ]:
% mkdir /content/data

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
import time
t0 = time.time()
gdd.download_file_from_google_drive(file_id='1K-vkjl94l2JUDsAG9njNt4_uMcxdkIjm',
                                    dest_path='/content/data/files.zip',
                                    unzip=True)

Unzipping...Done.


In [ ]:
% rm /content/data/files.zip

Next we load csv dataframes. 

In [ ]:
df_train_val = pd.read_csv('/content/data/trainset/trainset_true.csv')
df_test = pd.read_csv('/content/data/testset/testset_data.csv')

# Preprocess data

As described in our report , we will use clinical attributes along with images for our predictions. To accelerate convergence , we scale lymphocytes count and convert dates to timestamps. 

## birth dates preprocessing

In [ ]:
def date_to_float(x) :
      """
      Function that converts date string to datetime and then to timestamp.
      """
      try : 
          date_time_obj = datetime.strptime(x, '%d-%m-%Y')
      except :
          date_time_obj = datetime.strptime(x, '%m/%d/%Y')

      return date_time_obj.timestamp()


df_train_val['birth_float'] = df_train_val['DOB'].apply(date_to_float)
df_test['birth_float'] = df_test['DOB'].apply(date_to_float)

## scale with scklearn scaler 
x = df_train_val.birth_float.values.reshape(-1,1) #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_train_val['birth_float'] = x_scaled
df_test['birth_float'] = min_max_scaler.transform(df_test.birth_float.values.reshape(-1,1))

## Lymph count preprocessing

In [ ]:
x = df_train_val.LYMPH_COUNT.values.reshape(-1,1) #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_train_val['lymph'] = x_scaled
df_test['lymph'] = min_max_scaler.transform(df_test.LYMPH_COUNT.values.reshape(-1,1))

Next we will split train and validation sets . Since the negative class is rare , we will stratify our split based on the classes frequency. 

In [ ]:
df_train, df_val = train_test_split(df_train_val, test_size=0.2 , stratify = df_train_val['LABEL'])
df_train.reset_index(drop=True , inplace=True)
df_val.reset_index(drop=True , inplace=True)

# Custom dataset and dataloader

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df,path_images, transform=None):
        """
        This class creates torch custom dataset. 
        args :
          df : train or validation dataframe
          path_images : test or train images path.
          transform : torchvision transforms , different between train and validation/test. 
        """
        self.df_frame = df
        self.labels = self.df_frame['LABEL'].tolist()
        self.path_images = path_images
        self.transform = transform

    def __len__(self):
        return len(self.df_frame)

    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()

        id = self.df_frame.loc[idx, 'ID']

        ## get the directory of the bag number id . 
        dir =os.path.join(self.path_images,id)
        _, _, files = next(os.walk(dir))
        file_count = len(files)
        # create the batch of the instances.
        all_imgs = torch.zeros((file_count,3,224,224))

        for k,img_name in enumerate(os.listdir(dir)) :
            img_dir = os.path.join(dir,img_name)
            image = Image.open(img_dir)
            img_transformed = self.transform(image)
            all_imgs[k] = img_transformed
        label = self.df_frame.loc[idx, 'LABEL']
        # get clinical attributes
        clinical_attributes = torch.tensor([float(self.df_frame.loc[idx,'birth_float']),
                                            float(self.df_frame.loc[idx,'lymph'])])
        dict_res = {'image':all_imgs , 'label':label ,
                    'clinical':torch.FloatTensor(clinical_attributes) ,'ID':id}
        return dict_res

In [ ]:
path_images = '/content/data/trainset/'
path_images_test = '/content/data/testset/'

Next we apply transformations for the images : for train we randomly rotate and flip and then normalize , for validation and test we just normalize. 

In [ ]:
transform_train = transforms.Compose([transforms.RandomRotation(90),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip() , 
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225])])

transform_val = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])])

trainset = CustomDataset(df_train,path_images , transform=transform_train )
valset = CustomDataset(df_val ,path_images , transform=transform_val )
testset = CustomDataset(df_test ,path_images_test , transform=transform_val )


In [ ]:
train_loader = torch.utils.data.DataLoader(
          trainset, batch_size=1, shuffle=True, num_workers=2, drop_last=False)

validation_loader = torch.utils.data.DataLoader(
          valset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)

test_loader = torch.utils.data.DataLoader(
          testset, batch_size=1, shuffle=False, num_workers=2, drop_last=False)

# Model

In the next cell we define the model architecture . It is composed of two classes : a subclass where we define the attention network and the main class where we define the whole model.

In [ ]:
class MOE(nn.Module):
    def __init__(self):
        super().__init__()

        ## MLP for clinical attributes 
        self.mlp1 = nn.Linear(2,2)
        self.mlp2 = nn.Linear(2,1)

        ## images features part 
        self.pretrained_model  = models.alexnet(pretrained=False)
        self.attn = AttentionSoftMax(in_features=1000) ## call attention
        self.cnn_class = nn.Linear(1000 , 1)

        self.relu = nn.ReLU()

    def forward(self, x , clinical ):
        
        # Images predictions
        x = torch.squeeze(x, dim=0) 
        features = self.pretrained_model(x)
        features = self.attn(features).unsqueeze(0)
        pred_im = self.cnn_class(features)

        # clinical features predictions
        out_mlp = self.relu(self.mlp1(clinical))
        pred_mlp = self.mlp2(out_mlp)

        final_output = (pred_im+pred_mlp)/2
        return  final_output
      
class AttentionSoftMax(torch.nn.Module):
    """
    take from : https://medium.com/swlh/multiple-instance-learning-c49bd21f5620.
    """
    def __init__(self, in_features = 1000, out_features = None):
        """
        given a tensor `x` with dimensions [N * M],
        where M -- dimensionality of the featur vector
                   (number of features per instance)
              N -- number of instances
        initialize with `AggModule(M)`
        returns:
        - weighted result: [M]
        - gate: [N]
        """
        super(AttentionSoftMax, self).__init__()
        self.otherdim = ''
        if out_features is None:
            out_features = in_features
        self.layer_linear_tr = nn.Linear(in_features, out_features)
        self.activation = nn.LeakyReLU()
        self.layer_linear_query = nn.Linear(out_features, 1)
        self.sig = nn.Sigmoid()
        
    def forward(self, x):
        keys = self.layer_linear_tr(x)
        keys = self.activation(keys) 
        attention_map_raw = self.layer_linear_query(keys)[...,0]
        attention_map = nn.Softmax(dim=-1)(attention_map_raw)
        result = torch.einsum(f'{self.otherdim}i,{self.otherdim}ij->{self.otherdim}j', attention_map, x)
        return result

Next we define the model instance, the optimizer and the scheduler. 

In [ ]:
epochs = 200


model = MOE()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=  1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, patience=20, factor=.5, threshold=1e-4, verbose=True)


# Train

In the next two cells we define functions for training and validation. Then we run training for 200 epochs while saving the best checkpoints.

In [ ]:

def train(epoch):
    """
    Function for training the model.
    """
    print('\nEpoch: %d' % epoch)
    print('Train ...')
    model.train()
    train_loss  = 0 

    for batch_idx, data in tqdm(enumerate(train_loader),total = len(train_loader)):
        image , label = data['image'].to(device),data['label'].type(torch.FloatTensor).unsqueeze(dim=0).to(device)
        clinical_attributes = data['clinical'].type(torch.FloatTensor).to(device)
        optimizer.zero_grad()
        outputs= model(image,clinical_attributes)
        loss = nn.BCEWithLogitsLoss()(outputs, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    #print('Loss: %.3f'% (train_loss/(batch_idx+1)))


def eval_epoch():
    """
    Function for evaluation. 
    """
    model.eval()
    print('Validate ...')
    y_true , y_preds , val_loss_all = [] , [] , 0
    with torch.no_grad():
        for batch_idx, data in tqdm(enumerate(validation_loader),total = len(validation_loader)):
      
            image , label = data['image'].to(device),data['label'].type(torch.FloatTensor).unsqueeze(dim=0).to(device)
            clinical_attributes = data['clinical'].type(torch.FloatTensor).to(device)
            y_true.append(int(label.item()))
            outputs = model(image,clinical_attributes)
            val_loss = nn.BCEWithLogitsLoss()(outputs, label)
            ## predict here 
            y_pred = int(torch.round(torch.sigmoid(outputs.cpu()))[0].item())
            y_preds.append(y_pred)

            val_loss_all +=val_loss.item()

    ## compute all metrics and then take the mean 
    balanced = balanced_accuracy_score (y_true , y_preds )
    acc = accuracy_score(y_true,y_preds)
    avg_prec = average_precision_score (y_true , y_preds)
    mean_scores =(balanced + acc + avg_prec)/3
    return val_loss_all/(batch_idx+1) , balanced , mean_scores


 

In [ ]:
min_val_loss = np.inf

for epoch in range(epochs):
    train(epoch)
    val_loss , score_balanced , mean_scores  = eval_epoch()
    scheduler.step(val_loss)
    
    # save checkpoint only the mean of metrics is higher than the current max. 
    if val_loss < min_val_loss : 
        min_val_loss = val_loss
        torch.save(model.state_dict(), '/content/MIL_state.pth')
    
    print('balanced accuracy on val : ',score_balanced)
    print('val loss ',val_loss)
    print('mean scores ',mean_scores)
  


Epoch: 0
Train ...



balanced accuracy on val :  0.5
val loss  0.6211387352509932
mean scores  0.6313131313131314

Epoch: 1
Train ...



balanced accuracy on val :  0.5
val loss  0.6099216983173833
mean scores  0.6313131313131314

Epoch: 2
Train ...



balanced accuracy on val :  0.5
val loss  0.6131942868232727
mean scores  0.6313131313131314

Epoch: 3
Train ...



balanced accuracy on val :  0.5
val loss  1.296219832263887
mean scores  0.6313131313131314

Epoch: 4
Train ...


...


balanced accuracy on val :  0.7478260869565218
val loss  0.4731390726837245
mean scores  0.7685258380910555

Epoch: 196
Train ...



balanced accuracy on val :  0.75
val loss  0.3789270856615269
mean scores  0.8066378066378066

Epoch: 197
Train ...



balanced accuracy on val :  0.7478260869565218
val loss  0.43497713039996044
mean scores  0.7685258380910555

Epoch: 198
Train ...



balanced accuracy on val :  0.75
val loss  0.36465175370826863
mean scores  0.8066378066378066

Epoch: 199
Train ...



balanced accuracy on val :  0.85
val loss  0.3598975076368361
mean scores  0.8812354312354312


# Submit 

In [ ]:
def submit():
    model.eval()
    y_preds  =  [] 
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader):
            image = data['image'].to(device)
            clinical_attributes = data['clinical'].type(torch.FloatTensor).to(device)
            outputs = model(image,clinical_attributes)
            y_pred = int(torch.round(torch.sigmoid(outputs.cpu()))[0].item())
            y_preds.append(y_pred)

    return  y_preds

In [ ]:
model = MOE()
model.to(device)
model.load_state_dict(torch.load('/content/MIL_state.pth'))
y_preds = submit()

In [ ]:
submission = pd.DataFrame(list(zip(df_test['ID'], y_preds)), columns = ['Id', 'Predicted'])
submission.to_csv('/content/submission_x.csv',index=False)